# [admin] Instancetypes command


The `instancetypes` command in `admin` scope could help you manage instancetypes.


## Setup PrimeHub Python SDK


In [1]:
from primehub import PrimeHub, PrimeHubConfig
ph = PrimeHub(PrimeHubConfig())

if ph.is_ready():
    print("PrimeHub Python SDK setup successfully")
else:
    print("PrimeHub Python SDK couldn't get the group information, follow the 00-getting-started.ipynb to complete it")

PrimeHub Python SDK setup successfully


## Help documentation

In [2]:
help(ph.admin.instancetypes)

Help on AdminInstanceTypes in module primehub.admin_instancetypes object:

class AdminInstanceTypes(primehub.Helpful, primehub.Module)
 |  AdminInstanceTypes(primehub: primehub.PrimeHub, **kwargs)
 |  
 |  Method resolution order:
 |      AdminInstanceTypes
 |      primehub.Helpful
 |      primehub.Module
 |      primehub.HTTPSupport
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  add_group(self, id: str, group_id)
 |      Add group connection to an instance type by id.
 |      
 |      :type id: str
 |      :param id: the id of an instance type
 |      :type group_id: str
 |      :param group_id: group id
 |      
 |      :rtype dict
 |      :return an instance type with groups only
 |  
 |  create(self, config)
 |      Create an instance type
 |      
 |      :type config: dict
 |      :param config: The configurations for creating an instance type
 |      
 |      :rtype dict
 |      :return The instance type
 |  
 |  delete(self, id)
 |      Delete an instance type 

## InstanceTypes management

In [3]:
!primehub admin instancetypes help

Usage: 
  primehub admin instancetypes <command>

Manage instance type

Available Commands:
  add-group            Add group connection to an instance type by id
  create               Create an instance type
  delete               Delete an instance type by id
  get                  Get an instance type by id
  list                 List instance type
  list-groups          List groups of an instance type by id
  remove-group         Remove group connection from an instance type by id
  update               Update the instance type

Options:
  -h, --help           Show the help

Global Options:
  --config CONFIG      Change the path of the config file (Default: ~/.primehub/config.json)
  --endpoint ENDPOINT  Override the GraphQL API endpoint
  --token TOKEN        Override the API Token
  --group GROUP        Override the current group
  --json               Output the json format (output human-friendly format by default)


### Fields for creating or updating

| field | required | type | description |
| --- | --- | --- | --- |
| name | required | string | it should be a valid resource name for kubernetes. `name` will be ignored when updating |
| displayName | optional | string | display name for the instance type |
| description | optional | string | |
| global | optional | boolean | when an instance type is global, it could be seen for each group |
| groups | optional | list of connected groups (dict) | please see the `connect` examples. default value: false |
| cpuLimit | required* | float | the maximum vCPU quantity. For example: `1` or `1.0` means 1 vCPU and `0.5` means half of vCPU |
| cpuRequest | optional | float | the initial vCPU quantity for CPU resource. cpuRequest can not be greater than cpuLimit |
| memoryLimit | required* | float | the maximum Memory size. For example: `1.5` means `1.5 GB` memory |
| memoryRequest | optional | float | the initial Memory size. memoryRequest can not be greater than memoryLimit |
| gpuLimit | optional | int | the count of GPU when an instance allocated |
| tolerations | optional | dict | kubernetes pod toleration in an instance (Pod) |
| nodeSelector | optional | dict | kubernetes pod nodeSelector in an instance (Pod) |

#### Auto-filling Fields
Auto-filling will happen when the configuration omitted fields

| field	| value	| description |
| --- | --- | --- |
| cpuLimit	| 1	| 1 vCPU |
| memoryLimit	| 1	| 1 GB memory |


## Examples

You could find [more examples on our github](https://github.com/InfuseAI/primehub-python-sdk/blob/main/docs/CLI/admin/instancetypes.md).

In [4]:
import pandas as pd

#### CRUD: instance type

In [5]:
# create an instance type
config = {
  "name": "cpu-instance-by-sdk",
  "displayName": "CPU 1",
  "description": "1 vCPU / 1G Memory",
  "cpuLimit": 1,
  "memoryLimit": 1,
  "gpuLimit": 0,
  "global": True
}
data = ph.admin.instancetypes.create(config)

In [6]:
# List users
instance_type_df = pd.DataFrame(ph.admin.instancetypes.list())
instance_type_df[instance_type_df['name'] == config['name']]

,id,name,displayName,description,cpuLimit,memoryLimit,gpuLimit
6,cpu-instance-by-sdk,cpu-instance-by-sdk,CPU 1,1 vCPU / 1G Memory,1.0,1,0


In [7]:
# get deatils by id
ph.admin.instancetypes.get(data['id'])

{'id': 'cpu-instance-by-sdk',
 'cpuRequest': None,
 'memoryRequest': None,
 'global': True,
 'groups': [],
 'tolerations': [],
 'nodeSelector': None,
 'name': 'cpu-instance-by-sdk',
 'displayName': 'CPU 1',
 'description': '1 vCPU / 1G Memory',
 'cpuLimit': 1,
 'memoryLimit': 1,
 'gpuLimit': 0}

In [8]:
# Update the user config setting.
update_config = {"memoryLimit": 2}
data = ph.admin.instancetypes.update(data['id'], update_config)
ph.admin.instancetypes.get(data['id'])

{'id': 'cpu-instance-by-sdk',
 'cpuRequest': None,
 'memoryRequest': None,
 'global': True,
 'groups': [],
 'tolerations': [],
 'nodeSelector': {},
 'name': 'cpu-instance-by-sdk',
 'displayName': 'CPU 1',
 'description': '1 vCPU / 1G Memory',
 'cpuLimit': 1,
 'memoryLimit': 2,
 'gpuLimit': 0}

In [9]:
# delete the instance type
ph.admin.instancetypes.delete(data['id'])

{'id': 'cpu-instance-by-sdk'}

#### CRUD: instance type <-> Group

In [10]:
# create an instance type
config = {
  "name": "cpu-instance-by-sdk-group",
  "displayName": "CPU 1",
  "description": "1 vCPU / 1G Memory",
  "cpuLimit": 1,
  "memoryLimit": 1,
  "gpuLimit": 0,
  "global": False
}
data = ph.admin.instancetypes.create(config)

In [11]:
# Show the instance type group list

instance_type_group_df = pd.DataFrame(ph.admin.instancetypes.list_groups(data["id"]))
instance_type_group_df

""


In [12]:
# Add the instance type into group
groups_df = pd.DataFrame(ph.admin.groups.list())
groups_id = groups_df["id"][0]
ph.admin.instancetypes.add_group(data["id"], groups_id)

In [13]:
# Show the instance type group list

user_group_df = pd.DataFrame(ph.admin.instancetypes.list_groups(data["id"]))
user_group_df

,id,name,displayName
0,71ac1e32-65fa-4e8e-a735-ba282e3149b1,AnotherTestGroup,Another Test Group


In [14]:
# Remove the instance type out of group
ph.admin.instancetypes.remove_group(data["id"], groups_id)

In [15]:
# Show the instance type group list

user_group_df = pd.DataFrame(ph.admin.instancetypes.list_groups(data["id"]))
user_group_df

""


In [16]:
# Delete the instance type
ph.admin.instancetypes.delete(data['id'])

{'id': 'cpu-instance-by-sdk-group'}